In [5]:
from __future__ import division
from __future__ import print_function
import time
import tensorflow as tf
import numpy as np


In [10]:
#set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

import sys, os
import numpy as np
import time
import csv
from sklearn.metrics import confusion_matrix
import scipy.io as sio
import random
import ABIDE_Parser as Reader
import keras

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
import pickle as pkl 
import time
import copy
import scipy.spatial.distance
from tqdm import tqdm
from tensorflow.python.ops import array_ops



Some functions in Tensorflow 1.x

In [11]:
def glorot(shape, name=None):
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    var = tf.Variable(initial, name=name)
    return var
def zeros(shape, name=None):
    """All zeros."""
    initial = tf.zeros(shape, dtype=tf.float32)
    return tf.Variable(initial, name=name)

def dot(x, y, sparse=False):
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x,y)
    else:
        res = tf.matmul(x,y)
    return res

def accuracy(preds, labels):
    correct_prediction = tf.equal(tf.round(preds), labels)
    accuracy = tf.cast(correct_prediction, tf.float32)
    return tf.reduce_mean(accuracy)

def tens(shape, name=None):
    initial = tf.constant(10, tf.float32, shape)
    return tf.Variable(initial, name=name)


In [12]:
class gat_layer(object):
    def __init__(self, input_dim,F_, placeholders,attn_heads=1,attn_heads_reduction='concat',
                 activation=tf.nn.relu, use_bias=True,name_=''):
        self.dropout_rate = placeholders['dropout']
        self.in_drop = placeholders['in_drop']
        self.name = 'gat_layer'+name_
        self.vars = {}
        self.act = activation
        self.attn_heads = attn_heads  # Number of attention heads (K in the paper)
        self.attn_heads_reduction = attn_heads_reduction  #
        self.bias = use_bias
        self.A = placeholders["adj"]
        self.input_dim = input_dim

        with tf.variable_scope(self.name+'_vars'):
            for i in range(self.attn_heads):
                self.vars['weights_'+str(i)] = glorot([input_dim, F_], name='weights_' + str(i))
                self.vars["attn_self_weights_"+str(i)] = glorot([F_, 1], name='attn_self_weights_' + str(i))
                self.vars["attn_neighs_weights_"+str(i)] = glorot([F_, 1], name='attn_neighs_weights_' + str(i))
        if self.bias:
            self.vars['bias'] = zeros([F_],name='bias')
            
    def __call__(self, inputs):
        X = inputs
        if self.in_drop != 0.0:
            X = tf.nn.dropout(X, 1-self.in_drop)
        outputs = []
        dense_mask = []
        
        for head in range(self.attn_heads):
            # Compute inputs to attention network
            kernel = self.vars['weights_'+str(head)]
            features = tf.tensordot(X, kernel, axes=1)  # (N x F')

            # Compute feature combinations
            attention_self_kernel = self.vars["attn_self_weights_"+str(head)]
            attention_neighs_kernel = self.vars["attn_neighs_weights_"+str(head)]
            attn_for_self = tf.tensordot(features, attention_self_kernel, axes=1)    
            attn_for_neighs = tf.tensordot(features, attention_neighs_kernel, axes=1)  

            # Attention head a(Wh_i, Wh_j) = a^T [[Wh_i], [Wh_j]]
            dense = attn_for_self + tf.transpose(attn_for_neighs, [0,2,1])  # (N x N) via broadcasting
            
            #print("plus:", dense.shape)

            # Add nonlinearty
            dense = tf.nn.leaky_relu(dense,alpha=0.2)

            zero_vec = -9e15*tf.ones_like(dense)
            dense = tf.where(self.A > 0.0, dense, zero_vec)
            dense_mask.append(dense)

            # Apply softmax to get attention coefficients
            dense = tf.nn.softmax(dense)  # (N x N)

            # Apply dropout to features and attention coefficients
            dropout_attn = tf.nn.dropout(dense, 1-self.dropout_rate) # (N x N)
            dropout_feat = tf.nn.dropout(features, 1-self.dropout_rate)  # (N x F')

            # Linear combination with neighbors' features
            node_features = tf.matmul(dropout_attn, dropout_feat)  # (N x F')

            if self.bias:
                node_features += self.vars["bias"]

            # Add output of attention head to final output
            if self.attn_heads_reduction == 'concat':
                outputs.append(self.act(node_features))
            else:
                outputs.append(node_features)

        # Aggregate the heads' output according to the reduction method
        if self.attn_heads_reduction == 'concat':
            output = tf.concat(outputs, axis=-1)  # (N x KF')
        else:
            output = tf.add_n(outputs) / self.attn_heads  # N x F')
            output = self.act(output)

        return output, dense_mask


In [13]:
class fc_layer(object):

    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,sparse_input=False, act=tf.nn.relu, bias=False, featureless=False,name_=''):
        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.name = 'fc_layer'+name_
        self.vars = {}
        self.act = act

        self.sparse_input = sparse_input
        self.featureless = featureless
        self.bias = bias

        with tf.variable_scope(self.name+'_vars'):
            self.vars['weights'] = glorot([input_dim, output_dim], name='weights')
        if self.bias:
            self.vars['bias'] = zeros([output_dim],name='bias')

    def __call__(self, inputs):
        x = inputs

        x = tf.nn.dropout(x,1-self.dropout)

        output = tf.tensordot(x, self.vars['weights'], axes=1)

        if self.bias:
            output += self.vars['bias']
        return self.act(output)

In [14]:
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_integer('node_num', 110, 'Number of Graph nodes')

flags.DEFINE_integer('output_dim', 1, 'Number of output_dim')
flags.DEFINE_float('learning_rate', 0.0001, 'Initial learning rate') #0.0005，0.0001，0.00005，0.00001，0.00003
flags.DEFINE_integer('batch_num', 10, 'Number of epochs to train')
flags.DEFINE_integer('epochs', 1000, 'Number of epochs to train')
flags.DEFINE_integer('attn_heads', 5, 'Number of attention head')

flags.DEFINE_integer('hidden1_gat', 24, 'Number of units in hidden layer 1 of gcn')
flags.DEFINE_integer('output_gat', 3, 'Number of units in output layer 1 of gcn')

flags.DEFINE_float('dropout', 0, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('in_drop', 0, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 15, 'Tolerance for early stopping (# of epochs).')


In [15]:
class Model(object):

    def __init__(self, placeholders, input_dim):
        self.placeholders = placeholders
        self.input_dim = input_dim
        self.name = 'gat_mil'

        self.gat_layers = []
        self.fc_layers = []
        self.gcn_activations = []
        self.fc_activatinos = []

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        self.outputs = None
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]

        self.loss = 0
        self.accuracy = 0
        
        self.node_prob = None
        self.dense_mask = []

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
        self.opt_op = None
        
        self.loss_explainer = 0
        self.optimizer_explainer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
        self.opt_op_explainer = None
        self.M = tens((FLAGS.node_num, FLAGS.node_num), name='mask')
        
        self.build()

    def build(self):
        with tf.variable_scope(self.name):
            self._build()

        sigmoid_M = tf.sigmoid(self.M)
        self.inputs = tf.multiply(self.inputs, sigmoid_M)

        self.gcn_activations.append(self.inputs)

        for layer in self.gat_layers:
            hidden, dense_mask = layer(self.gcn_activations[-1])
            self.gcn_activations.append(hidden) 
            self.dense_mask.append(dense_mask)


        p_layer = self.fc_layers[0]
        node_prob = p_layer(self.gcn_activations[-1])
        
        tensor = tf.reshape(node_prob, shape=(-1, FLAGS.node_num))
        layer = self.fc_layers[1]
        attention_prob = layer(tensor)


        attention_mul = tf.multiply(tensor, attention_prob)
        self.outputs = tf.reduce_sum(attention_mul, 1, keep_dims=True)
        print(self.outputs.shape)

        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        self._loss()
        self._accuracy()


        var_list = tf.trainable_variables()
        var_list1 = []
        for var in var_list:
            if var != self.M:
                var_list1.append(var)
            elif var == self.M:
                #stop = input("M exit!!!!!!!")
                pass
        
        self.opt_op = self.optimizer.minimize(self.loss, var_list = [var_list1])
        self.loss_explainer += tf.reduce_mean(tf.losses.log_loss(labels=self.placeholders['labels'], predictions=self.outputs))
        self.opt_op_explainer = self.optimizer_explainer.minimize(self.loss_explainer, var_list=[self.M])

    def _build(self):
        self.gat_layers.append(gat_layer(input_dim=self.input_dim,F_=FLAGS.hidden1_gat, placeholders=self.placeholders,
                                         attn_heads=FLAGS.attn_heads,attn_heads_reduction='concat',
                                         activation=tf.nn.leaky_relu, use_bias=True,name_='1'))

        self.gat_layers.append(gat_layer(input_dim=FLAGS.hidden1_gat*FLAGS.attn_heads,F_=FLAGS.output_gat, placeholders=self.placeholders,
                                         attn_heads=3,attn_heads_reduction='average',
                                         activation=tf.nn.leaky_relu, use_bias=True,name_='2'))

        self.fc_layers.append(fc_layer(input_dim=FLAGS.output_gat, output_dim=FLAGS.output_dim, placeholders=self.placeholders, 
                                       act=tf.nn.sigmoid, dropout=True, name_='1'))
        
        self.fc_layers.append(fc_layer(input_dim=FLAGS.node_num, output_dim=FLAGS.node_num, placeholders=self.placeholders, 
                                       act=tf.nn.softmax, dropout=True, name_='2'))

    def _loss(self):
        for var in self.gat_layers[0].vars.values():
            self.loss += FLAGS.weight_decay*tf.nn.l2_loss(var)

            
        self.loss += tf.reduce_mean(tf.losses.log_loss(labels=self.placeholders['labels'], predictions=self.outputs))

    def _accuracy(self):
        self.accuracy = accuracy(self.outputs, self.placeholders['labels'])

    def predict(self):
        return self.outputs